PROCESS DOCUMENT

In [127]:
import os
from IPython.display import Markdown
from phi.agent import Agent
from phi.model.ollama import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from utils.document_processor import DocumentProcessor  

# Inisialisasi path dan model
DATA_PATH = "./data"
INDEX_PATH = "faiss_index"
CHUNKED_DATA_PATH = "./chunked_data"  # Folder untuk menyimpan hasil chunking
OLLAMA_MODEL = "llama3.2"

# Pastikan folder tersedia
os.makedirs(CHUNKED_DATA_PATH, exist_ok=True)

# Inisialisasi model Ollama
llm = Ollama(id=OLLAMA_MODEL)

# Inisialisasi embedding model menggunakan HuggingFace
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Inisialisasi document processor untuk ekstraksi teks
docs = DocumentProcessor()

# Fungsi untuk chunking teks secara efisien
def chunk_text(text: str):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, length_function=len, separators=["\n\n", "\n", " "]
)
    return text_splitter.split_text(text)

# Inisialisasi Agent dengan model Ollama
agent = Agent(model=llm, show_tool_calls=True, markdown=True)

# Langkah 1: Proses dokumen dengan agentic chunking
extracted_docs = []
for filename in os.listdir(DATA_PATH):
    valid_extensions = ('.pdf', '.docx', '.txt')
    if not filename.lower().endswith(valid_extensions):
        continue

    filepath = os.path.join(DATA_PATH, filename)

    try:
        # Gunakan DocumentProcessor untuk membaca file
        with open(filepath, "rb") as f:
            document = f.read()
            result = docs.process_document(document, filename)

        if result is None:
            print(f"Gagal memproses {filename}, melewati file ini.")
            continue

        plain_text = result[3]  # Pastikan indeks benar

        # Gunakan agent untuk memastikan chunking lebih semantik
        response = agent.run(
            f"Organize the following text into well-structured segments:\n{plain_text}",
            max_tokens=4000  # Atur sesuai dengan batas model
        )

        # Proses hasil agen dengan chunking tambahan
        chunks = chunk_text(str(response))  

        # Simpan hasil chunking dalam list
        extracted_docs.extend([{"text": chunk, "metadata": {"filename": filename}} for chunk in chunks])

        # Simpan hasil chunking dalam file txt
        chunked_filepath = os.path.join(CHUNKED_DATA_PATH, f"chunked_{filename}.txt")
        with open(chunked_filepath, "w", encoding="utf-8") as chunked_file:
            for i, chunk in enumerate(chunks, 1):
                chunked_file.write(f"Chunk {i}:\n{chunk}\n\n")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Simpan hasil chunking sebagai objek Document
documents = [Document(page_content=doc["text"], metadata=doc["metadata"]) for doc in extracted_docs]

# Simpan ke FAISS secara lokal
vector_store = FAISS.from_documents(documents, embedding_model)
vector_store.save_local(INDEX_PATH)


print(f"Total chunks generated: {len(chunks)} for {filename}")
print("Proses chunking selesai. Hasilnya disimpan dalam folder 'chunked_data'.")


Total chunks generated: 12 for Dokumen (1).pdf
Proses chunking selesai. Hasilnya disimpan dalam folder 'chunked_data'.


In [128]:
# Query dengan RAG
query = "Apa isi dokumen tentang topik X?"
query_embedding = embedding_model.embed_query(query)
retrieved_docs = vector_store.similarity_search_by_vector(query_embedding, k=3)

# Gabungkan dokumen hasil pencarian untuk input ke model
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

TEST OLLAMA MODEL

In [129]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [130]:
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vectore_test"

In [131]:
template = """
  Anda adalah asisten yang membantu dalam meringkas teks.  
  Hanya sertakan informasi yang ada dalam dokumen.  
  Jangan tambahkan opini atau analisis Anda sendiri.  

  Dokumen:  
  "{document}"  
  Ringkasan:  
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model=OLLAMA_MODEL)

chain = prompt | model

response = chain.invoke({"document": plain_text})
Markdown(response)


Berikut ringkasan dari dokumen tersebut:

PT Pertamina (Persero) atau "PERTAMINA" memiliki sejarah yang panjang dimulai pada tahun 1950-an. PERTAMINA dipimpin oleh PT Eksploitasi Tambang Minyak Sumatera Utara, kemudian menjadi Perusahaan Minyak Nasional (PERMINA), dan setelah itu menjadi Perusahaan Negara (PN) dengan nama PN Pertambangan Minyak Nasional (Permina). Pada tahun 1968, PN Permina bergabung dengan PN Pertamin sehingga terbentuk PN Pertambangan Minyak dan Gas Bumi Negara (Pertamina).

Dalam beberapa tahun kemudian, PERTAMINA mengalami perubahan nama menjadi Perusahaan Pertambangan Minyak dan Gas Bumi Negara, kemudian menjadi PT Pertamina (Persero) pada tahun 2003. Pada tahun 2007, PERTAMINA mengubah visinya menjadi "Menjadi Perusahaan Minyak Nasional Kelas Dunia", kemudian pada tahun 2011 menjadi "Menjadi Perusahaan Energi Nasional Kelas Dunia".

PERTAMINA telah mencapai beberapa keberhasilan, seperti menuntaskan akuisisi saham perusahaan migas Prancis Maurel et Prom (M&P) di tahun 2017 dan mengakuisi 51% saham PT Pertamina Gas (Pertagas) oleh PT Perusahaan Gas Negara (PGN) pada tahun 2018.

Dalam beberapa tahun terakhir, PERTAMINA telah melakukan restrukturisasi pembentukan subholding, yang dimulai dari pembentukan Subholding Gas pada tahun 2018 dan diselesaikan pada tanggal 1 September 2021. Dengan demikian, PERTAMINA dapat lebih fokus untuk mewujudkan kedaulatan energi bagi Indonesia dengan senantiasa menggaungkan semangat "One Energy, One Pertamina".

TEST OPENAI MODEL

In [132]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-35-turbo"
response = openai.ChatCompletion.create(
  engine=model,
  messages=[
    {
      "role": "system", 
      "content": """
        You are a helpful assistant for text summarization.
        Only include information that is part of the document. 
        Do not include your own opinion or analysis.
      """
    },
    {
      "role": "user",
      "content": plain_text
    }
  ],
)

Markdown(response.choices[0].message.content)

PT Pertamina is an energy company established by the Indonesian government in the 1950s, initially managing oil fields in Sumatra. Over the years, the company has undergone several name and status changes, becoming a state-owned enterprise and officially adopting the name PT Pertamina in 2003. In recent years, PT Pertamina has worked towards realizing its vision of becoming a world-class energy company. In 2017, PT Pertamina acquired a 72.65% stake in French oil company Maurel et Prom, giving it a presence in 12 countries across four continents. Additionally, in 2018, PT Pertamina's subsidiary, Pertamina Gas, was acquired by state-owned company Perusahaan Gas Negara, further solidifying PT Pertamina's position as a leading entity in Indonesia's energy sector. The formation of several sub-holdings in 2020 and 2021 aims to further sharpen PT Pertamina's focus to achieve energy sovereignty for Indonesia.